In [24]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import numpy
import requests
from WalkScore import *
import copy

In [3]:
cab_data_p1 = pd.read_csv("cab_data_p1.csv")
cab_data_p2 = pd.read_csv("cab_data_p2.csv")

In [29]:
#REMOVE WHEN UPLOADING
apiKey= ''
walkscore = WalkScore(apiKey)

#Requires pip install geopy
def get_address(df):
    geolocator = Nominatim()
    
    def f(x,y):
        coord = "%f, %f" % (x, y)
        return coord
    
    def get_d(x,y):
        return geodesic(x, y).miles
    
    def reverse_geocode(latlng):
        res = geolocator.reverse(latlng)
        return res.address
    
    
    df['start_latlng'] = df[['dropoff_latitude', 'dropoff_longitude']].apply(lambda x: f(*x), axis=1)
    df['end_latlng'] = df[['pickup_latitude', 'pickup_longitude']].apply(lambda x: f(*x), axis=1)
    
    #VERY SLOW
    df['pickup_address'] = df['start_latlng'].apply(reverse_geocode)
    df['dropoff_address'] = df['end_latlng'].apply(reverse_geocode)
    df['distance'] = df[['start_latlng', 'end_latlng']].apply(lambda x: get_d(*x), axis=1)
    
    
    
    #head = df.head(n=50)
    #print(head)
    #head['test'] = head['start_latlng'].apply(reverse_geocode)
    #print(head['test'])
    #head['distance'] = head[['start_latlng', 'end_latlng']].apply(lambda x: get_d(*x), axis=1)
    #print(head['distance'])
    #print(df['start_latlng'])
    #location = geolocator.reverse("")
    #print("getting")
    
    res_df = df[['dropoff_latitude', 'dropoff_longitude', 'pickup_latitude', 'pickup_longitude', 'distance', 'start_latlng', 'end_latlng', 'pickup_address', 'dropoff_address']]
    
    return res_df
    
    
    

#Given part 1 and part 2 of data, returns a DF of walkability score for 
#Start and end coord
def get_walkability_score(df):
    def ws_score(lat,lon, address):
        score = walkscore.makeRequest(address, lat, lon)
        return score['walkscore']
    
    start_df = df[['pickup_latitude', 'pickup_longitude', 'pickup_address']].apply(lambda x: ws_score(*x), axis=1)
    end_df = df[['dropoff_latitude', 'dropoff_longitude', 'dropoff_address']].apply(lambda x: ws_score(*x), axis=1)
    return start_df, end_df

In [33]:
print("Getting addresses")
res_df = get_address(copy.copy(cab_data_p1[:20]))
#print(res_df)
#print(res_df['pickup_address'][0])
a = 40.722809 
b= -74.005013
c = res_df['pickup_address'][0]

Getting addresses


In [34]:
print("Getting walkability scores")
start_w, end_w = get_walkability_score(res_df)
print(start_w)
print(end_w)

Getting walkability scores
0     100
1     100
2     100
3      99
4     100
5      99
6      99
7      98
8      99
9     100
10     98
11     98
12     98
13     57
14    100
15     99
16     99
17     99
18     99
19     98
dtype: int64
0      99
1      93
2     100
3      99
4     100
5      99
6      99
7      99
8      99
9     100
10     99
11     97
12     99
13     99
14     98
15    100
16    100
17     99
18     65
19     98
dtype: int64
